# **5. 분류와 군집화로 이해하는 지도 학습과 비지도 학습**
---
* 출판사 : 생능 출판사( http://www.booksr.co.kr/ )
* 으뜸 파이썬 저자 : 강영민, 박동규, 김성수
*  소스코드 저장소 : https://github.com/dknife/ML2nd
*  저작권 : 본 노트북 코드는 자유롭게 배포가능하지만 위의 출판사, 저서, 저자표기와 함께 배포해 주십시오.
---


### LAB5-1 붓꽃 데이터를 군집화하도록 하자


### **5장 미니 프로젝트 A1** - 잡음제거: k-NN의 활용



In [3]:
# 필요한 라이브러리 불러오기
from sklearn import cluster                  # K-평균 군집화 모듈
from sklearn.datasets import load_iris       # 아이리스 데이터셋
from sklearn.metrics import accuracy_score   # 정확도 계산
import numpy as np                           # 수치 연산용

# 아이리스 데이터 불러오기
iris = load_iris()
# iris.data   → 150개의 샘플 × 4개 특성 (꽃받침 길이/너비, 꽃잎 길이/너비)
# iris.target → 실제 정답 레이블 (0: setosa, 1: versicolor, 2: virginica)
# iris.target_names → ['setosa', 'versicolor', 'virginica']

# 군집 수(k)를 실제 품종 수와 동일하게 설정
k = len(iris.target_names)    # 3 (품종이 3개이므로 군집도 3개로)
print(f'군집 수 k = {k}')

군집 수 k = 3


In [4]:
print('KMeans() 군집화 적용...')
# K-평균 모델 생성 및 학습
model = cluster.KMeans(n_clusters=k,          # 군집 개수 = 3
                       random_state=42)       # 결과 재현을 위해 seed 고정
                                              # 특성 데이터만으로 학습 (비지도)
model.fit(iris.data)

# 각 샘플이 어느 군집에 속하는지 예측
labels = model.predict(iris.data)
print('군집화 결과 labels :', labels[:150])   # 실제 품종과 다를 수 있다

# 실제 품종별로 군집 결과가 어떻게 나뉘었는지 확인해보자
#    아이리스는 0~49: setosa, 50~99: versicolor, 100~149: virginica
a_counts = np.bincount(labels[:50])      # setosa 구간의 군집 번호 카운트
b_counts = np.bincount(labels[50:100])   # versicolor 구간
c_counts = np.bincount(labels[100:])     # virginica 구간

print("setosa 구간에서 각 군집 번호 개수     :", a_counts)
print("versicolor 구간에서 각 군집 번호 개수 :", b_counts)
print("virginica 구간에서 각 군집 번호 개수  :", c_counts)

KMeans() 군집화 적용...
군집화 결과 labels : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 0 0 0 2 0 0 0 0
 0 0 2 2 0 0 0 0 2 0 2 0 2 0 0 2 2 0 0 0 0 0 2 0 0 0 0 2 0 0 0 2 0 0 0 2 0
 0 2]
setosa 구간에서 각 군집 번호 개수     : [ 0 50]
versicolor 구간에서 각 군집 번호 개수 : [ 3  0 47]
virginica 구간에서 각 군집 번호 개수  : [36  0 14]


In [6]:
# 올바른 매핑: 군집 번호 → 실제 품종
mapping = {}
mapping[np.argmax(a_counts)] = 0   # 가장 많이 나온 군집 = setosa
mapping[np.argmax(b_counts)] = 1   # 가장 많이 나온 군집 = versicolor
mapping[np.argmax(c_counts)] = 2   # 가장 많이 나온 군집 = virginica

print("군집 : 실제 품종 매핑:", mapping)
# 예: {1: 0, 2: 1, 0: 2} → 군집1=setosa, 군집2=versicolor, 군집0=virginica


# 새로운 레이블 생성
new_labels = np.array([mapping[label] for label in labels])
print("새로운 labels :", new_labels[:150])

군집 : 실제 품종 매핑: {np.int64(1): 0, np.int64(2): 1, np.int64(0): 2}
새로운 labels : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 2 1 2 2 2 2
 2 2 1 1 2 2 2 2 1 2 1 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2 2 1 2 2 2 1 2 2 2 1 2
 2 1]


In [7]:
# 군집화 성능 평가 (실제 정답과 비교해서 정확도 계산)
accuracy = accuracy_score(iris.target, new_labels)
print(f'아이리스 데이터의 군집화 정확도: {accuracy:.4f} ({accuracy*100:.2f}%)')
# 보통 0.89 ~ 0.96 정도

아이리스 데이터의 군집화 정확도: 0.8867 (88.67%)
